# DataHacks2020: Science Track

## Exploratory Data Analysis

### Notes:
* Five or more relatied sections from any topic of interest
* NLP on questions
* 2001-2016 time span
* srat cat 2 and 3 are all null
* response is not response its null?, DataValue contains the values
* plotly

In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv('data/U.S._Chronic_Disease_Indicators__CDI_.csv')

In [3]:
raw_data.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2015,2015,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_2,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,11,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,66,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2012,2012,PR,Puerto Rico,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,72,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2012,2012,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN


In [4]:
# attributes to note: YearStart/YearEnd, Tpoic, Question, Response, DataValue, Geolocation, StartificationCategory
raw_data.describe()

,YearStart,YearEnd,DataValueAlt,LowConfidenceLimit,HighConfidenceLimit,LocationID
count,519718.000000,519718.000000,3.503350e+05,311062.000000,311062.000000,519718.000000
mean,2013.141885,2013.162754,8.917742e+02,46.759401,58.991287,30.993144
std,1.777622,1.762672,1.833073e+04,77.492628,88.668862,17.723341
min,2001.000000,2001.000000,0.000000e+00,0.200000,0.420000,1.000000
25%,2012.000000,2012.000000,1.845500e+01,12.700000,18.900000,17.000000
50%,2013.000000,2013.000000,4.100000e+01,30.200000,43.800000,30.000000
75%,2015.000000,2015.000000,7.030000e+01,55.400000,70.400000,45.000000
max,2016.000000,2016.000000,2.600878e+06,1330.660000,2088.000000,78.000000


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519718 entries, 0 to 519717
Data columns (total 34 columns):
YearStart                    519718 non-null int64
YearEnd                      519718 non-null int64
LocationAbbr                 519718 non-null object
LocationDesc                 519718 non-null object
DataSource                   519718 non-null object
Topic                        519718 non-null object
Question                     519718 non-null object
Response                     79188 non-null object
DataValueUnit                476046 non-null object
DataValueType                519718 non-null object
DataValue                    374564 non-null object
DataValueAlt                 350335 non-null float64
DataValueFootnoteSymbol      227318 non-null object
DatavalueFootnote            227152 non-null object
LowConfidenceLimit           311062 non-null float64
HighConfidenceLimit          311062 non-null float64
StratificationCategory1      519718 non-null object
Strat

Response, ResponseID, Stratification2/3, StratificationID2/3, StratificationCategory2/3, and StratificationCategoryID2/3 all have ~ 79000 non-null, which is suspicious

In [7]:
null_cols = ['Response', 'ResponseID', 'Stratification2', 'Stratification3', 
             'StratificationCategory2', 'StratificationCategory3', 'StratificationID2', 'StratificationID3', 
             'StratificationCategoryID2', 'StratificationCategoryID3']

for col in null_cols:
    print(raw_data[col].value_counts())

     79188
Name: Response, dtype: int64
     79176
Name: ResponseID, dtype: int64
     79188
Name: Stratification2, dtype: int64
     79188
Name: Stratification3, dtype: int64
     79188
Name: StratificationCategory2, dtype: int64
     79188
Name: StratificationCategory3, dtype: int64
     79189
Name: StratificationID2, dtype: int64
     79188
Name: StratificationID3, dtype: int64
     79189
Name: StratificationCategoryID2, dtype: int64
     79188
Name: StratificationCategoryID3, dtype: int64


All of these columns are blank/null, so we can safely remove them.

In [8]:
data = raw_data.drop(null_cols, axis=1)

There is both a Topic and TopicID column, which could be redundant.

In [9]:
data[['Topic', 'TopicID']].groupby('Topic')['TopicID'].nunique()

Topic
Alcohol                                            1
Arthritis                                          1
Asthma                                             1
Cancer                                             1
Cardiovascular Disease                             1
Chronic Kidney Disease                             1
Chronic Obstructive Pulmonary Disease              1
Diabetes                                           1
Disability                                         1
Immunization                                       1
Mental Health                                      1
Nutrition, Physical Activity, and Weight Status    1
Older Adults                                       1
Oral Health                                        1
Overarching Conditions                             1
Reproductive Health                                1
Tobacco                                            1
Name: TopicID, dtype: int64

Indeed, Topic and TopicID are redundant, so we can remove one. In this case, TopicID is mor ambiguous, so we remove it. However, we will keep a map for reference.

In [10]:
TopicIdToTopic = data[['Topic', 'TopicID']].drop_duplicates().set_index('TopicID')
data = data.drop('TopicID', axis=1)

Let's now look at the topics:

In [11]:
data.Topic.value_counts(normalize=True)

Diabetes                                           0.153220
Chronic Obstructive Pulmonary Disease              0.151484
Cardiovascular Disease                             0.145823
Arthritis                                          0.080361
Overarching Conditions                             0.075737
Asthma                                             0.075543
Nutrition, Physical Activity, and Weight Status    0.064691
Alcohol                                            0.061283
Tobacco                                            0.056388
Older Adults                                       0.029439
Cancer                                             0.027132
Chronic Kidney Disease                             0.023849
Oral Health                                        0.021429
Mental Health                                      0.013902
Immunization                                       0.010044
Disability                                         0.005711
Reproductive Health                     

There is the most data for Diabetes, so we'll explore it first.

In [12]:
diabetes = data.loc[data.Topic=='Diabetes']
diabetes.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,...,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
258,2012,2012,IL,Illinois,BRFSS,Diabetes,Foot examination among adults aged >= 18 years...,%,Age-adjusted Prevalence,NaN,...,NaN,NaN,Gender,Male,"(40.485010283, -88.997710178)",17,DIA5_0,AGEADJPREV,GENDER,GENM
262,2011,2011,IL,Illinois,BRFSS,Diabetes,Foot examination among adults aged >= 18 years...,%,Crude Prevalence,NaN,...,NaN,NaN,Gender,Male,"(40.485010283, -88.997710178)",17,DIA5_0,CRDPREV,GENDER,GENM
276,2010,2010,AK,Alaska,NVSS,Diabetes,Mortality due to diabetes reported as any list...,"cases per 100,000",Age-adjusted Rate,51.9,...,42.1,61.7,Gender,Female,"(64.845079957001, -147.722059036)",2,DIA1_1,AGEADJRATE,GENDER,GENF
303,2011,2011,KS,Kansas,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,Number,27,...,NaN,NaN,Overall,Overall,"(38.3477403, -98.200781227)",20,DIA1_2,NMBR,OVERALL,OVR
308,2016,2016,FL,Florida,BRFSS,Diabetes,Influenza vaccination among noninstitutionaliz...,%,Crude Prevalence,48,...,42.9,53.2,Race/Ethnicity,"White, non-Hispanic","(28.932040377, -81.928960539)",12,DIA12_1,CRDPREV,RACE,WHT


In [15]:
diabetes.iloc[10]

YearStart                                                                 2010
YearEnd                                                                   2010
LocationAbbr                                                                AR
LocationDesc                                                          Arkansas
DataSource                                                                NVSS
Topic                                                                 Diabetes
Question                     Mortality due to diabetes reported as any list...
DataValueUnit                                                              NaN
DataValueType                                                           Number
DataValue                                                                 1091
DataValueAlt                                                              1091
DataValueFootnoteSymbol                                                       
DatavalueFootnote                                   

In [18]:
diabetes.Question.value_counts()

Mortality due to diabetes reported as any listed cause of death                                        6135
Mortality with diabetic ketoacidosis reported as any listed cause of death                             6135
Prevalence of depressive disorders among adults aged >= 18 years with diagnosed diabetes               5220
Adults with diagnosed diabetes aged >= 18 years who have taken a diabetes self-management course       5220
Dilated eye examination among adults aged >= 18 years with diagnosed diabetes                          5220
Influenza vaccination among noninstitutionalized adults aged >= 65 years with diagnosed diabetes       5220
Influenza vaccination among noninstitutionalized adults aged 18-64 years with diagnosed diabetes       5220
Pneumococcal vaccination among noninstitutionalized adults aged >= 65 years with diagnosed diabetes    5220
Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes              5220
Pneumococcal vaccination amo

In [21]:
diabetes.loc[diabetes.Question=='Mortality due to diabetes reported as any listed cause of death']

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,...,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
276,2010,2010,AK,Alaska,NVSS,Diabetes,Mortality due to diabetes reported as any list...,"cases per 100,000",Age-adjusted Rate,51.9,...,42.1,61.7,Gender,Female,"(64.845079957001, -147.722059036)",2,DIA1_1,AGEADJRATE,GENDER,GENF
366,2010,2010,AR,Arkansas,NVSS,Diabetes,Mortality due to diabetes reported as any list...,NaN,Number,1091,...,NaN,NaN,Gender,Female,"(34.748650124, -92.274490743)",5,DIA1_1,NMBR,GENDER,GENF
422,2012,2012,WV,West Virginia,NVSS,Diabetes,Mortality due to diabetes reported as any list...,"cases per 100,000",Crude Rate,,...,NaN,NaN,Race/Ethnicity,Hispanic,"(38.665510202, -80.712640135)",54,DIA1_1,CRDRATE,RACE,HIS
444,2014,2014,AL,Alabama,NVSS,Diabetes,Mortality due to diabetes reported as any list...,"cases per 100,000",Age-adjusted Rate,,...,NaN,NaN,Race/Ethnicity,Asian or Pacific Islander,"(32.840571122, -86.631860762)",1,DIA1_1,AGEADJRATE,RACE,API
472,2010,2010,WY,Wyoming,NVSS,Diabetes,Mortality due to diabetes reported as any list...,NaN,Number,306,...,NaN,NaN,Race/Ethnicity,"White, non-Hispanic","(43.235541343, -108.109830353)",56,DIA1_1,NMBR,RACE,WHT
482,2011,2011,FL,Florida,NVSS,Diabetes,Mortality due to diabetes reported as any list...,NaN,Number,1499,...,NaN,NaN,Race/Ethnicity,Hispanic,"(28.932040377, -81.928960539)",12,DIA1_1,NMBR,RACE,HIS
712,2012,2012,CO,Colorado,NVSS,Diabetes,Mortality due to diabetes reported as any list...,NaN,Number,141,...,NaN,NaN,Race/Ethnicity,"Black, non-Hispanic","(38.843840757, -106.133610921)",8,DIA1_1,NMBR,RACE,BLK
785,2013,2013,MS,Mississippi,NVSS,Diabetes,Mortality due to diabetes reported as any list...,"cases per 100,000",Age-adjusted Rate,144.1,...,135.9,152.2,Race/Ethnicity,"Black, non-Hispanic","(32.745510099, -89.538030825)",28,DIA1_1,AGEADJRATE,RACE,BLK
907,2014,2014,AK,Alaska,NVSS,Diabetes,Mortality due to diabetes reported as any list...,NaN,Number,,...,NaN,NaN,Race/Ethnicity,Hispanic,"(64.845079957001, -147.722059036)",2,DIA1_1,NMBR,RACE,HIS
977,2012,2012,OH,Ohio,NVSS,Diabetes,Mortality due to diabetes reported as any list...,"cases per 100,000",Age-adjusted Rate,77.1,...,75.1,79.0,Gender,Female,"(40.060210141, -82.404260056)",39,DIA1_1,AGEADJRATE,GENDER,GENF


In [22]:
diabetes.loc[diabetes.Question=='Mortality with diabetic ketoacidosis reported as any listed cause of death']

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,...,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
303,2011,2011,KS,Kansas,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,Number,27,...,NaN,NaN,Overall,Overall,"(38.3477403, -98.200781227)",20,DIA1_2,NMBR,OVERALL,OVR
664,2010,2010,RI,Rhode Island,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,Number,,...,NaN,NaN,Gender,Female,"(41.708280193, -71.522470314)",44,DIA1_2,NMBR,GENDER,GENF
676,2012,2012,OR,Oregon,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,"cases per 100,000",Age-adjusted Rate,0.9,...,0.7,1.3,Overall,Overall,"(44.567449424, -120.155031326)",41,DIA1_2,AGEADJRATE,OVERALL,OVR
703,2014,2014,AK,Alaska,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,"cases per 100,000",Crude Rate,,...,NaN,NaN,Race/Ethnicity,American Indian or Alaska Native,"(64.845079957001, -147.722059036)",2,DIA1_2,CRDRATE,RACE,AIAN
828,2011,2011,CT,Connecticut,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,"cases per 100,000",Crude Rate,,...,NaN,NaN,Gender,Female,"(41.56266102, -72.649840952)",9,DIA1_2,CRDRATE,GENDER,GENF
1390,2011,2011,AR,Arkansas,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,"cases per 100,000",Age-adjusted Rate,1.4,...,0.8,2.1,Gender,Male,"(34.748650124, -92.274490743)",5,DIA1_2,AGEADJRATE,GENDER,GENM
1610,2014,2014,US,United States,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,Number,3889,...,NaN,NaN,Overall,Overall,NaN,59,DIA1_2,NMBR,OVERALL,OVR
1636,2011,2011,KS,Kansas,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,Number,,...,NaN,NaN,Gender,Female,"(38.3477403, -98.200781227)",20,DIA1_2,NMBR,GENDER,GENF
1816,2012,2012,UT,Utah,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,Number,,...,NaN,NaN,Race/Ethnicity,"White, non-Hispanic","(39.360700171, -111.587130635)",49,DIA1_2,NMBR,RACE,WHT
2136,2011,2011,ID,Idaho,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,"cases per 100,000",Age-adjusted Rate,,...,NaN,NaN,Gender,Female,"(43.682630005, -114.363730042)",16,DIA1_2,AGEADJRATE,GENDER,GENF


It appears that each question corresponds to its own QuestionID.  
For example, DIA1_1 corresponse to : 'Mortality due to diabetes reported as any listed cause of death'

In [19]:
# most common questions
data.Question.value_counts()

Hospitalization for hip fracture among Medicare-eligible persons aged >= 65 years                                                       7488
Hospitalization for chronic obstructive pulmonary disease as first-listed diagnosis among Medicare-eligible persons aged >= 65 years    7488
Hospitalization for heart failure among Medicare-eligible persons aged >= 65 years                                                      7488
Hospitalization for chronic obstructive pulmonary disease as any diagnosis among Medicare-eligible persons aged >= 65 years             7488
Asthma mortality rate                                                                                                                   6135
Mortality from diseases of the heart                                                                                                    6135
Mortality from cerebrovascular disease (stroke)                                                                                         6135
Mortality wit